# Desafio Prático

## Objetivos de Ensino

Exercitar os seguintes conceitos trabalhados no Módulo:

    ✔ Exercitar o módulo Spark SQL do Apache Spark;
    ✔ Exercitar o módulo Spark MLLib do Apache Spark.

# Enunciado

Doenças ligadas ao coração afetam milhões de pessoas ao redor do mundo e, segundo a Organização Mundial da Saúde (OMS), são a segunda principal causa de
morte da população mundial. Como cientista de dados, você foi contratado para criar um modelo preditivo que, a partir de dados de pacientes como idade, gênero, nível de glicose, se o paciente fuma ou não, vai prever se aquele paciente terá um derrame cerebral ou não.

Você tem acesso a um arquivo que possui atributos de pacientes e um atributo “stroke” (derrame), que indica se aquele paciente sofreu um evento de derrame ou não.

O conjunto de dados está disponível em:

●  https://dcc.ufmg.br/~pcalais/stroke_data.csv

Para uma descrição das colunas, veja a seção “Attribute information” em:

●  https://www.kaggle.com/fedesoriano/stroke-prediction-dataset

In [3]:
from pyspark.sql import SparkSession

# Spark entry point
spark = SparkSession \
    .builder \
    .appName("Desafio pratico 2 - Cientista de Dados - Apache Spark") \
    .getOrCreate()

In [5]:
spark.version

'3.3.1'

In [6]:
# Carregando a planilha CSV

pacientes_df = spark.read.csv('stroke_data.csv',header='True',inferSchema='True')

In [7]:
# Detalhando os atributos do data frame

pacientes_df.printSchema()

root
 |-- 0: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- age: double (nullable = true)
 |-- hypertension: integer (nullable = true)
 |-- heart_disease: integer (nullable = true)
 |-- ever_married: string (nullable = true)
 |-- work_type: string (nullable = true)
 |-- Residence_type: string (nullable = true)
 |-- avg_glucose_level: double (nullable = true)
 |-- bmi: double (nullable = true)
 |-- smoking_status: string (nullable = true)
 |-- stroke: integer (nullable = true)



In [8]:
# Visualizando os 10 primeiros dados do df

pacientes_df.show(10)

+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  0|gender| age|hypertension|heart_disease|ever_married|    work_type|Residence_type|avg_glucose_level|  bmi| smoking_status|stroke|
+---+------+----+------------+-------------+------------+-------------+--------------+-----------------+-----+---------------+------+
|  1|Female|18.0|           0|            0|          No|      Private|         Urban|            94.19|12.12|         smokes|     1|
|  2|  Male|58.0|           1|            0|         Yes|      Private|         Rural|           154.24| 33.7|   never_smoked|     0|
|  3|Female|36.0|           0|            0|         Yes|     Govt_job|         Urban|            72.63| 24.7|         smokes|     0|
|  4|Female|62.0|           0|            0|         Yes|Self-employed|         Rural|            85.52| 31.2|formerly smoked|     0|
|  5|Female|82.0|           0|            0|         Yes|     

### Quantos registros existem no arquivo?

In [9]:
  # extraindo quantidade de linhas
  linhas = pacientes_df.count()
   
  # extraindo quantidade de colunas
  colunas = len(pacientes_df.columns)
 
  # print
  print(f'A dimensão da planilha é: {(linhas,colunas)}')
  print(f'Quantidade de linhas: {linhas}')
  print(f'Quantidade de colunas: {colunas}')

A dimensão da planilha é: (67135, 12)
Quantidade de linhas: 67135
Quantidade de colunas: 12


### No conjunto de dados, quantos pacientes sofreram e não sofreram derrame (stroke)?

In [10]:
sDerrame = pacientes_df.filter(pacientes_df["stroke"] == 1).count()
nDerrame = pacientes_df.filter(pacientes_df["stroke"] == 0).count()

print("Número de pacientes que sofreram derrame e não sofreram derrame, respectivamete:", sDerrame, "e", nDerrame)

Número de pacientes que sofreram derrame e não sofreram derrame, respectivamete: 40287 e 26848


###  A partir do dataframe, crie uma tabela temporária usando df.createOrReplaceTempView('table') e a seguir use spark.sql para escrever uma consulta SQL que obtenha quantos pacientes tiveram derrame por tipo de trabalho (work_type).

In [11]:
pacientes_df.createOrReplaceTempView("table")

consultaSQL = """
SELECT work_type, SUM(stroke) as stroke_count
FROM table
GROUP BY work_type
"""

pacientes = spark.sql(consultaSQL)
pacientes.show()

+-------------+------------+
|    work_type|stroke_count|
+-------------+------------+
| Never_worked|          85|
|Self-employed|       10807|
|      Private|       23711|
|     children|         520|
|     Govt_job|        5164|
+-------------+------------+



### Escreva uma consulta com spark.sql para determinar a proporção, por gênero, de participantes do estudo.

In [12]:
novaConsulta = """
SELECT gender, COUNT(*) as total, COUNT(*)/(SELECT COUNT(*) FROM table) as proporcao
FROM table
GROUP BY gender
"""

genero = spark.sql(novaConsulta)
genero.show()

+------+-----+--------------------+
|gender|total|           proporcao|
+------+-----+--------------------+
|Female|39530|  0.5888135845684069|
| Other|   11|1.638489610486333...|
|  Male|27594|  0.4110225664705444|
+------+-----+--------------------+



### Escreva uma consulta com spark.sql para determinar quem tem mais probabilidade de sofrer derrame: hipertensos ou não-hipertensos.

In [13]:
consultaHipertenso = """
SELECT hypertension, COUNT (*) as hypertension_count, SUM(stroke) as stroke_count, SUM(stroke)/COUNT(*) as stroke_probabilidade
FROM table
GROUP BY hypertension
"""

hipertensos = spark.sql(consultaHipertenso)
hipertensos.show()

+------------+------------------+------------+--------------------+
|hypertension|hypertension_count|stroke_count|stroke_probabilidade|
+------------+------------------+------------+--------------------+
|           1|             11017|        8817|  0.8003086139602432|
|           0|             56118|       31470|  0.5607826365871913|
+------------+------------------+------------+--------------------+



### Escreva uma consulta com spark.sql que determine o número de pessoas que sofreram derrame por idade.

In [22]:
ConsultaDerrame = """
SELECT age, SUM(stroke) as stroke_count
FROM table
GROUP BY age
ORDER BY stroke_count desc
"""

DerrameIdade = spark.sql(ConsultaDerrame)
DerrameIdade.show()


+----+------------+
| age|stroke_count|
+----+------------+
|79.0|        2916|
|78.0|        2279|
|80.0|        1858|
|81.0|        1738|
|82.0|        1427|
|77.0|         994|
|74.0|         987|
|63.0|         942|
|76.0|         892|
|70.0|         881|
|66.0|         848|
|75.0|         809|
|67.0|         801|
|57.0|         775|
|73.0|         759|
|65.0|         716|
|72.0|         709|
|68.0|         688|
|69.0|         677|
|71.0|         667|
+----+------------+
only showing top 20 rows



### Usando a API de dataframes, determine quantas pessoas sofreram derrames após os 50 anos.

In [18]:
print("Número de pessoas que sofreram derrames após os 50 anos:", pacientes_df.filter((pacientes_df["stroke"] == 1) & (pacientes_df["age"] > 50)).count())

Número de pessoas que sofreram derrames após os 50 anos: 28938


###  Usando spark.sql, determine qual o nível médio de glicose para pessoas que, respectivamente, sofreram e não sofreram derrame.

In [21]:
consultaGlicose = """
SELECT stroke, ROUND(AVG(avg_glucose_level),2) as media_glucose
FROM table
GROUP BY stroke
"""

glicose = spark.sql(consultaGlicose)
glicose.show()

+------+-------------+
|stroke|media_glucose|
+------+-------------+
|     1|       119.95|
|     0|        103.6|
+------+-------------+



### Qual é o BMI (IMC = índice de massa corpórea) médio de quem sofreu e não sofreu derrame?

In [24]:
consultaIMC = """
SELECT stroke, ROUND(AVG(bmi),2) as media_imc
FROM table
GROUP BY stroke
"""

imc = spark.sql(consultaIMC)
imc.show()

+------+---------+
|stroke|media_imc|
+------+---------+
|     1|    29.94|
|     0|    27.99|
+------+---------+

